In [9]:
library(tidyverse)
library(CellChat)
library(repr)
library(scales)
library(writexl)

In [2]:
dsID=str_c("ds",1:50,sep = "")

In [ ]:
dsID

In [4]:
sampleID=c("148_T1","148_T3","142_T5-4","132_T1","142_T2","18_T3","18_T4","17_T4","151_T4","WT1","WT2")

In [5]:
ds_sample=tibble(ds=rep(dsID,each = length(sampleID)),
                sample=rep(sampleID,length(dsID)))

In [6]:
longLRtable=function(DS,SAMPLE){
    file=str_c("/data2/xiangwei/projects/zhaolian/16cellchat/cellchatEpidownsample/",
           DS,"/results/",SAMPLE,"count.csv",sep="")
    LRtable=read.table(file,sep = ",")
    LRtable=LRtable%>%rownames_to_column(var="celltype1")%>%pivot_longer(-celltype1,names_to = "celltype2",values_to = "LRnum")%>%
mutate(dsID=DS,sampleID=SAMPLE)
}

In [7]:
LRalltable=map2_dfr(ds_sample$ds,ds_sample$sample,longLRtable)

In [8]:
dim(LRalltable)
head(LRalltable)
table(LRalltable$sampleID)

[1] 38818     5

celltype1,celltype2,LRnum,dsID,sampleID
<chr>,<chr>,<int>,<chr>,<chr>
Epi-Goblet,Epi.Goblet,3,ds1,148_T1
Epi-Goblet,Epi.Mdga2,8,ds1,148_T1
Epi-Goblet,Epi.Mgst3,0,ds1,148_T1
Epi-Goblet,Epi.Mt3,1,ds1,148_T1
Epi-Goblet,Epi.Peak1,5,ds1,148_T1
Epi-Goblet,Epi.Ptprd,16,ds1,148_T1



  132_T1   142_T2 142_T5-4   148_T1   148_T3   151_T4    17_T4    18_T3 
    4050     4050     4050     3200     4050     3065     3287     3185 
   18_T4      WT1      WT2 
    4981     2450     2450 

In [10]:
write_xlsx(LRalltable,"results/LRalltable.xlsx")

# make LRtable

## wtLRtable

In [151]:
wtLRtable=LRalltable%>%filter(sampleID %in% c("WT1","WT2"))
dim(wtLRtable)

[1] 4900    5

## earlyLRtable

In [111]:
earlyLRtable=LRalltable%>%filter(sampleID %in% c("132_T1","142_T2","18_T3","18_T4","17_T4","151_T4"))
dim(earlyLRtable)

[1] 22618     5

## lateLRtable

In [153]:
lateLRtable=LRalltable%>%filter(sampleID %in% c("148_T1","148_T3","142_T5-4"))
dim(lateLRtable)

[1] 11300     5

In [212]:
df=wtLRtable

In [223]:
df=earlyLRtable

In [234]:
df=lateLRtable

In [235]:
head(df)

celltype1,celltype2,LRnum,dsID,sampleID
<chr>,<chr>,<int>,<chr>,<chr>
Epi-Goblet,Epi.Goblet,3,ds1,148_T1
Epi-Goblet,Epi.Mdga2,8,ds1,148_T1
Epi-Goblet,Epi.Mgst3,0,ds1,148_T1
Epi-Goblet,Epi.Mt3,1,ds1,148_T1
Epi-Goblet,Epi.Peak1,5,ds1,148_T1
Epi-Goblet,Epi.Ptprd,16,ds1,148_T1


In [236]:
df=df%>%mutate(celltype=str_c(celltype1,celltype2,sep = "_"))

In [237]:
avgTable=df%>%group_by(celltype)%>%summarise(n=n(),LRavg=mean(LRnum))

In [238]:
avgTable=avgTable%>%separate(celltype,into = c("celltype1","celltype2"),sep = "_")%>%select(-n)

In [239]:
widerAvgTable=avgTable%>%pivot_wider(names_from = celltype2,values_from = LRavg)

In [240]:
widerAvgTable=widerAvgTable%>%column_to_rownames("celltype1")

In [241]:
celltypeID=rownames(widerAvgTable)

In [242]:
colnames(widerAvgTable)=celltypeID

In [243]:
widerAvgTable=as.matrix(widerAvgTable)

In [222]:
write.table(widerAvgTable,"results/WTwiderAvgLRTable.csv",sep = ",")

In [233]:
write.table(widerAvgTable,"results/earlyWiderAvgLRTable.csv",sep = ",")

In [244]:
write.table(widerAvgTable,"results/lateWiderAvgLRTable.csv",sep = ",")

## persampleLRtable

In [11]:
persampleLRtable=vector("list",length = length(sampleID))
names(persampleLRtable)=sampleID

In [12]:
for(i in seq_along(sampleID)){
df=LRalltable%>%filter(sampleID %in% !!sampleID[i])
df=df%>%mutate(celltype=str_c(celltype1,celltype2,sep = "_"))
avgTable=df%>%group_by(celltype)%>%summarise(n=n(),LRavg=sum(LRnum)/50)
avgTable=avgTable%>%separate(celltype,into = c("celltype1","celltype2"),sep = "_")%>%select(-n)
widerAvgTable=avgTable%>%pivot_wider(names_from = celltype2,values_from = LRavg,values_fill=0)
widerAvgTable=widerAvgTable%>%column_to_rownames("celltype1")
celltypeID=rownames(widerAvgTable)
colnames(widerAvgTable)=celltypeID
widerAvgTable=as.matrix(widerAvgTable)
persampleLRtable[[i]]=widerAvgTable
    }

In [13]:
persampleLRtable[[1]]

,Epi-Goblet,Epi-Mdga2,Epi-Mgst3,Epi-Mt3,Epi-Peak1,Epi-Ptprd,Epi-Saa1,Epi-Srgn
Epi-Goblet,3.66,7.60,1.82,0.76,5.72,17.56,5.60,7.06
Epi-Mdga2,9.56,20.14,9.08,4.90,14.76,30.78,17.24,8.80
Epi-Mgst3,3.32,6.28,4.38,2.44,5.06,13.56,6.34,6.80
Epi-Mt3,2.76,6.08,4.16,4.06,5.14,11.68,5.94,7.28
Epi-Peak1,5.62,14.22,6.32,2.54,13.06,27.60,13.88,7.90
Epi-Ptprd,14.94,28.64,14.24,4.20,26.64,45.68,27.56,11.60
Epi-Saa1,10.84,21.34,10.48,2.54,19.84,40.14,21.16,12.66
Epi-Srgn,3.34,10.88,5.74,4.98,9.08,21.10,12.46,9.48


In [ ]:
avgLRnum=vector("numeric",length = length(sampleID))
for(i in seq_along(sampleID)){
avgLRnum[i]=sum(persampleLRtable[[i]])
    }
sampleID
avgLRnum

In [15]:
saveRDS(persampleLRtable,"results/persampleLRtable.rds")

# make cellnumTable

In [82]:
longCellNum=function(DS){
    file=str_c("/data2/xiangwei/projects/zhaolian/16cellchat/cellchatEpidownsample/",
           DS,"/results/","cellnum.csv",sep="")
    cellNumTable=read.table(file,sep = ",")
    cellNumTable=cellNumTable%>%rownames_to_column(var="celltype")%>%pivot_longer(-celltype,names_to = "sampleID",values_to = "cellnum")%>%
mutate(dsID=DS)
}

In [84]:
cellnumAllTable=map_dfr(dsID,longCellNum)

In [ ]:
#10celltype*11sample*50times=5500
dim(cellnumAllTable)
head(cellnumAllTable)

In [164]:
table(cellnumAllTable$sampleID)


      WT1       WT2   X132_T1   X142_T2 X142_T5.4   X148_T1   X148_T3   X151_T4 
      500       500       500       500       500       500       500       500 
   X17_T4    X18_T3    X18_T4 
      500       500       500 

## wtCellNumTable

In [162]:
wtCellNumTable=cellnumAllTable%>%filter(sampleID %in% c("WT1","WT2"))
dim(wtCellNumTable)

[1] 1000    4

## earlyCellNumTable

In [165]:
earlyCellNumTable=cellnumAllTable%>%filter(sampleID %in% c("X132_T1","X142_T2","X18_T3","X18_T4","X17_T4","X151_T4"))
dim(earlyCellNumTable)

[1] 3000    4

## lateCellNumTable

In [168]:
lateCellNumTable=cellnumAllTable%>%filter(sampleID %in% c("X148_T1","X148_T3","X142_T5.4"))
dim(lateCellNumTable)

[1] 1500    4

In [170]:
celldf=wtCellNumTable

In [175]:
celldf=earlyCellNumTable

In [180]:
celldf=lateCellNumTable

In [181]:
table(celldf$sampleID)


X142_T5.4   X148_T1   X148_T3 
      500       500       500 

In [182]:
avgCellTable=celldf%>%group_by(celltype)%>%summarise(n=n(),cellNumAvg=mean(cellnum))

In [174]:
write.table(avgCellTable,"results/WTavgCellNumTable.csv",sep = ",")

In [179]:
write.table(avgCellTable,"results/earlyAvgCellNumTable.csv",sep = ",")

In [184]:
write.table(avgCellTable,"results/lateAvgCellNumTable.csv",sep = ",")

## persampleCellNum

In [330]:
         #c("148_T1","148_T3","142_T5-4","132_T1","142_T2","18_T3","18_T4","17_T4","151_T4","WT1","WT2")
sampleIDX=c("X148_T1","X148_T3","X142_T5.4","X132_T1","X142_T2","X18_T3","X18_T4","X17_T4","X151_T4","WT1","WT2")

In [331]:
persampleCellNum=vector("list",length = length(sampleID))
names(persampleCellNum)=sampleID

In [332]:
for(i in seq_along(sampleID)){
celldf=cellnumAllTable%>%filter(sampleID %in% !!sampleIDX[i])
persampleCellNum[[i]]=celldf%>%group_by(celltype)%>%summarise(n=n(),cellNumAvg=mean(cellnum))
    }

#  plot

## max value

In [346]:
sampleGroupSize=vector("numeric",length = length(sampleID))
sampleCount=vector("numeric",length = length(sampleID))

In [351]:
for (i in seq_along(sampleID)) {
  sampleGroupSize[i]=max(persampleCellNum[[i]]$cellNumAvg)
  sampleCount[i]=max(persampleLRtable[[i]])
}
sampleID
max(sampleGroupSize)
max(sampleCount)

[1] "148_T1"   "148_T3"   "142_T5-4" "132_T1"   "142_T2"   "18_T3"   
 [7] "18_T4"    "17_T4"    "151_T4"   "WT1"      "WT2"

[1] 299

[1] 67.36

## plot

In [401]:
numClusterCheck=vector("numeric",length = 10)
for (i in c(1:7,9:11)) {
pdf(file = str_c("results/plot/",sampleID[i],"network.pdf"),height = 5,width = 5)
groupSize=persampleCellNum[[i]]%>%filter(cellNumAvg>6.89)
numClusterCheck[i]=identical(nrow(persampleLRtable[[i]]),nrow(groupSize))
netVisual_circle(persampleLRtable[[i]], vertex.weight = groupSize$cellNumAvg,idents.use = rownames(persampleLRtable[[i]]),
                 vertex.weight.max = 299, edge.weight.max = 67.36,
                 weight.scale = T, label.edge= F, 
                 )
dev.off() 
}

In [ ]:
pdf(file = str_c("results/plot/",sampleID[8],"network.pdf"),height = 5,width = 5)
groupSize=persampleCellNum[[8]]%>%filter(cellNumAvg>1)
netVisual_circle(persampleLRtable[[8]], vertex.weight = groupSize$cellNumAvg,idents.use = rownames(persampleLRtable[[8]]),
                 vertex.weight.max = 299, edge.weight.max = 67.36,
                 weight.scale = T, label.edge= F, 
                 )
dev.off()

In [412]:
save.image("results/episum_image")